# Run all the cells in this notebook!

In [ ]:
import itertools
import numpy as np
import pickle
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

## Generate one-hot encoding of 6-mers

In [ ]:
bases = ['G', 'C', 'T', 'A']

In [ ]:
possible_6_mers = sorted([
    ''.join(mer)
    for mer in list(itertools.product(bases, repeat = 6))
])

In [ ]:
def one_hot_6mer(sixmer):
    sixmer = sixmer.upper()
    index = sixmer_indices[sixmer]
    return [
        1 if i == index else 0
        for i in range(len(sixmer_indices))
    ]

def kmers(sequence, k):
    return [
        sequence[i:i+k]
        for i in range(len(sequence) - k + 1)
    ]

def encode_sequence(sequence):
    kmer_dict = {}
    for kmer in kmers(sequence, 6):
        if kmer in kmer_dict:
            kmer_dict[kmer] += 1
        else:
            kmer_dict[kmer] = 1
    
    return [
        kmer_dict[sixmer] if sixmer in kmer_dict else 0
        for sixmer in possible_6_mers
    ]

## Load labels

Expects a list of tuples of `(sequence, boolean for if it's an enhancer, list of tissues)`.

In [ ]:
with open('enhancer_features', 'rb') as f:
    labels = pickle.load(f)

## Specify enhancer prediction task

Specify what gets positive labels and what gets negative labels. Options are `forebrain`, `midbrain`, `limb`, `random`.

For example:
* Positive label `forebrain` with negative label `random` tries to classify whether a sequence is an enhancer in the forebrain region vs. anything else.
* Positive label `forebrain` with negative label `midbrain` tries to classify enhancers in the forebrain vs. enhancers in the midbrain. It will ignore any enhancers in both regions or sequences that aren't enhancers.

In [ ]:
def get_Y(enhancer, tissues, pos_label, neg_label):
    if pos_label in tissues and neg_label in tissues:
        return None
    if pos_label in tissues:
        return 1
    if neg_label in tissues:
        return 0
    if pos_label == 'random':
        return 1
    if neg_label == 'random':
        return 0
    return None
    
def get_X_Y(data, pos_label, neg_label):
    X = []
    Y = []
    
    for d in tqdm(data):
        y = get_Y(d[1], d[2], pos_label, neg_label)
        if y is not None:
            X.append(encode_sequence(d[0]))
            Y.append(y)
    
    return np.array(X), np.array(Y)

## Train and test an SVM model

In [ ]:
POSITIVE_LABEL = 'forebrain'
NEGATIVE_LABEL = 'random'

In [ ]:
X, Y = get_X_Y(labels, POSITIVE_LABEL, NEGATIVE_LABEL)

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
kf.get_n_splits(X)

In [ ]:
aurocs = []
for train_index, test_index in tqdm(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    clf = SVC(gamma='auto', probability=True)
    clf.fit(X_train, Y_train) 
    
    preds_with_scores = clf.predict_proba(X_test)
    aurocs.append(roc_auc_score(Y_test, preds_with_scores))
    
print('Average auROC: {}'.format(np.mean(aurocs)))

## TODO: generate precision recall curves (see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html)

## TODO: repeat the above five K fold training cells for midbrain vs. random, limb vs. random, forebrain vs. limb, midbrain vs. limb, forebrain vs. midbrain. Look at figure 2 in the paper.